In [1]:
'''
Objective:
•
Develop decision tree and random forest models to identify the key attributes contributing to high sales for a cloth manufacturing company, with Sales as the target variable, after converting it into a categorical variable.
Constraints:
•
Ensure proper preprocessing of the dataset, including handling missing values and encoding categorical variables, to prepare the data for model training while maintaining data integrity and quality.
•
Implement robust evaluation techniques to assess the performance of the decision tree and random forest models, considering metrics such as accuracy, precision, recall, and F1-score, and select the best-performing model for deployment.
Benefits/Impact:
•
The decision tree and random forest models provide insights into the significant attributes influencing high sales, enabling the cloth manufacturing company to prioritize investments and strategies for product development, marketing, and inventory management.
•
By deploying the final best solution with a user-friendly interface (UI), the company gains accessibility to actionable insights in real-time, facilitating data-driven decision-making and fostering agility in responding to market dynamics, ultimately leading to increased sales, profitability, and competitive advantage.


'''




import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from feature_engine.outliers import Winsorizer
import joblib
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as skmet


In [2]:
cloth = pd.read_csv('ClothCompany_data.csv')
cloth.head()

Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   

   Education Urban   US  
0         17   Yes  Yes  
1         10   Yes  Yes  
2         12   Yes  Yes  
3         14   Yes  Yes  
4         13   Yes   No

In [3]:
cloth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [4]:
cloth.describe()

Sales   CompPrice      Income  Advertising  Population  \
count  400.000000  400.000000  400.000000   400.000000  400.000000   
mean     7.496325  124.975000   68.657500     6.635000  264.840000   
std      2.824115   15.334512   27.986037     6.650364  147.376436   
min      0.000000   77.000000   21.000000     0.000000   10.000000   
25%      5.390000  115.000000   42.750000     0.000000  139.000000   
50%      7.490000  125.000000   69.000000     5.000000  272.000000   
75%      9.320000  135.000000   91.000000    12.000000  398.500000   
max     16.270000  175.000000  120.000000    29.000000  509.000000   

            Price         Age   Education  
count  400.000000  400.000000  400.000000  
mean   115.795000   53.322500   13.900000  
std     23.676664   16.200297    2.620528  
min     24.000000   25.000000   10.000000  
25%    100.000000   39.750000   12.000000  
50%    117.000000   54.500000   14.000000  
75%    131.000000   66.000000   16.000000  
max    191.000000   80.000000   18.000000

In [5]:
cloth.isna().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [6]:
x = cloth.drop(['Sales','Education','US'], axis =1)
y = cloth['Sales']

In [7]:
numeric_features = x.select_dtypes(exclude = ['object']).columns
numeric_features

Index(['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age'], dtype='object')

In [8]:
categorical_features = x.select_dtypes(include = ['object']).columns
categorical_features

Index(['ShelveLoc', 'Urban'], dtype='object')

In [9]:
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])
scale_pipeline = Pipeline(steps = [('scale', MinMaxScaler())])
encode_pipeline = Pipeline(steps = [('encode', OneHotEncoder(drop = 'first'))])

In [10]:
num_preprocessor = ColumnTransformer(transformers =[('num', num_pipeline, numeric_features)])
encode_preprocessor = ColumnTransformer(transformers=[('encode', encode_pipeline, categorical_features)])

In [11]:
impute_data = num_preprocessor.fit_transform(x)
encode_data = encode_preprocessor.fit_transform(x)


In [12]:
impute_df = pd.DataFrame(impute_data, columns = list(num_preprocessor.get_feature_names_out()))

In [13]:
encode_df = pd.DataFrame(encode_data, columns = list(encode_preprocessor.get_feature_names_out()))


In [14]:
scale_preprocessor = ColumnTransformer(transformers =[('scale', scale_pipeline, impute_df.columns)])

scale_data = scale_preprocessor.fit_transform(impute_df)

In [15]:
scale_df = pd.DataFrame(scale_data, columns = list(scale_preprocessor.get_feature_names_out()))


In [16]:
winsor = Winsorizer(capping_method = 'iqr',
                   tail = 'both',
                   fold = 1.5,
                   variables = list(scale_df.columns))

In [17]:
outlier_pipeline = Pipeline(steps = [('wins', winsor)])

In [18]:
outlier_data = outlier_pipeline.fit_transform(scale_df)

In [19]:
cleandata = pd.concat([outlier_data, encode_df], axis = 1)
cleandata

scale__num__CompPrice  scale__num__Income  scale__num__Advertising  \
0                 0.622449            0.525253                 0.379310   
1                 0.346939            0.272727                 0.551724   
2                 0.367347            0.141414                 0.344828   
3                 0.408163            0.797980                 0.137931   
4                 0.653061            0.434343                 0.103448   
..                     ...                 ...                      ...   
395               0.622449            0.878788                 0.586207   
396               0.632653            0.020202                 0.103448   
397               0.867347            0.050505                 0.413793   
398               0.234694            0.585859                 0.241379   
399               0.581633            0.161616                 0.000000   

     scale__num__Population  scale__num__Price  scale__num__Age  \
0                  0.533066           0.574850         0.309091   
1                  0.501002           0.353293         0.727273   
2                  0.519038           0.335329         0.618182   
3                  0.913828           0.437126         0.545455   
4                  0.661323           0.622754         0.236364   
..                      ...                ...              ...   
395                0.386774           0.622754         0.145455   
396                0.054108           0.574850         0.545455   
397                0.717435           0.808383         0.272727   
398                0.549098           0.425150         0.454545   
399                0.034068           0.574850         0.436364   

     encode__ShelveLoc_Good  encode__ShelveLoc_Medium  encode__Urban_Yes  
0                       0.0                       0.0                1.0  
1                       1.0                       0.0                1.0  
2                       0.0                       1.0                1.0  
3                       0.0                       1.0                1.0  
4                       0.0                       0.0                1.0  
..                      ...                       ...                ...  
395                     1.0                       0.0                1.0  
396                     0.0                       1.0                0.0  
397                     0.0                       1.0                1.0  
398                     0.0                       0.0                1.0  
399                     1.0                       0.0                1.0  

[400 rows x 9 columns]

In [20]:
joblib.dump(impute_data, 'impute_ensemble_assignment')
joblib.dump(scale_data, 'scale_ensemble_assignment')

joblib.dump(outlier_data, 'outlier_ensemble_assignment')

['outlier_ensemble_assignment']

In [21]:
#data splitting

In [22]:
x_train, x_test, y_train, y_test = train_test_split(cleandata, y, test_size = 0.2, random_state = 42)
x_train1 = x_train.astype('int')
x_test1 = x_test.astype('int')
y_train1 = y_train.astype('int')
y_test1 = y_test.astype('int')

In [23]:
dt_model = dt(criterion = 'entropy')

In [24]:
dt_model.fit(x_train1, y_train1)

DecisionTreeClassifier(criterion='entropy')

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [26]:
### Hyperparameter Optimization
# create a dictionary of all hyperparameters to be experimented


In [27]:
param_grid = {'criterion': ['gini','entropy'], 'max_depth': np.arange(3, 15)}

In [28]:
dt1_model = dt()

In [29]:
dt_cv = GridSearchCV(dt1_model, param_grid, cv = 5, scoring = 'accuracy', return_train_score = False)

In [30]:
dt_cv.fit(x_train1, y_train1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='accuracy')

In [31]:
dt_cv.best_params_

{'criterion': 'gini', 'max_depth': 3}

In [32]:
dt_cv.best_estimator_

DecisionTreeClassifier(max_depth=3)

In [33]:
pred = dt_cv.predict(x_test1)

In [34]:
accuracy_score(y_test1, pred)

0.1375

In [35]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [36]:
rf_model = rfc()

In [37]:
# #### Hyperparameters
# Number of trees in random forest


In [38]:
n_estimators = [int(x)for x in np.linspace(start = 10, stop = 100, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [3,6]
min_samples_split = [2,4]
min_samples_leaf = [1, 2]
bootstrap = [True, False]

In [39]:
rf_grid = {'n_estimators': n_estimators, 'max_features': max_features, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

In [40]:
rf_gd = GridSearchCV(estimator = rf_model, param_grid = rf_grid, cv = 10, n_jobs = -1)

In [41]:
rf_gd.fit(x_train1, y_train1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
1600 fits failed out of a total of 3200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1111 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._val

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [3, 6],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]})

In [42]:
rf_gd.best_params_

{'bootstrap': True,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 20}

In [43]:
accuracy_score(y_test1, rf_gd.predict(x_test1))

0.15

In [44]:
rf1_model = rfc(max_features = 'sqrt', max_depth = 3, min_samples_split = 2, min_samples_leaf = 1, bootstrap = True, n_estimators = 10)

In [45]:
rf1_model.fit(x_train1, y_train1)

RandomForestClassifier(max_depth=3, n_estimators=10)

In [46]:
accuracy_score(y_test1, rf1_model.predict(x_test1))

0.15

In [47]:
# Hyperparameter optimization with RandomizedSearchCV


In [48]:
rf_rs = RandomizedSearchCV(estimator = rf1_model, param_distributions = rf_grid, cv = 10)

In [49]:
rf_rs.fit(x_train1, y_train1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
40 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate

RandomizedSearchCV(cv=10,
                   estimator=RandomForestClassifier(max_depth=3,
                                                    n_estimators=10),
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [3, 6],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 4],
                                        'n_estimators': [10, 20, 30, 40, 50, 60,
                                                         70, 80, 90, 100]})

In [50]:
accuracy_score(y_test1, rf_rs.predict(x_test1))

0.1125

In [51]:
rf_rs.best_params_

{'n_estimators': 20,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 6,
 'bootstrap': False}

In [52]:
#baggingClassifier

In [53]:
from sklearn.ensemble import BaggingClassifier


In [54]:
bag_dt = BaggingClassifier(dt1_model, n_estimators = 10, bootstrap = True, random_state = 42)

In [55]:
bag_dt.fit(x_train1, y_train1)

BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

In [56]:
accuracy_score(y_test1, bag_dt.predict(x_test1))

0.1

In [57]:
bag_rf = BaggingClassifier(rf1_model, n_estimators = 60, bootstrap = False, random_state = 42)

In [58]:
bag_rf.fit(x_train1, y_train1)

BaggingClassifier(bootstrap=False,
                  estimator=RandomForestClassifier(max_depth=3,
                                                   n_estimators=10),
                  n_estimators=60, random_state=42)

In [59]:
accuracy_score(y_test1, bag_rf.predict(x_test1))

0.125

In [60]:
confusion_matrix(y_train1, bag_rf.predict(x_train1))

array([[ 0,  0,  0,  0,  0,  2,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0,  2,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  9,  0,  5,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1, 14,  1,  9,  8,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 19,  2, 16,  7,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  6,  4, 17,  9,  0,  2,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 10,  0, 20,  6,  0,  0,  8,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  2, 18,  9,  0,  5,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0, 11,  6,  1,  3,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  9,  3,  0,  7,  6,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  0,  1,  2,  0,  3, 10,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  3,  1,  0,  2,  4,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,

In [61]:
## Adaboosting

In [62]:
from sklearn.ensemble import AdaBoostClassifier

In [79]:
ada_clf = AdaBoostClassifier(learning_rate = 0.02, n_estimators = 100)

In [80]:
ada_clf.fit(x_train1, y_train1)

AdaBoostClassifier(learning_rate=0.02, n_estimators=100)

In [81]:
ada_pred = ada_clf.predict(x_test1)

In [82]:
accuracy_score(y_test1, ada_pred)

0.1125

In [83]:
#gradientboosting

In [84]:
from sklearn.ensemble import GradientBoostingClassifier

In [88]:
gb_clf = GradientBoostingClassifier(learning_rate = 0.02, n_estimators = 60, max_depth = 3)

In [89]:
gb_clf.fit(x_train1, y_train1)

GradientBoostingClassifier(learning_rate=0.02, n_estimators=60)

In [92]:
accuracy_score(y_test1,  gb_clf.predict(x_test1))

0.1

In [95]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test1, gb_clf.predict(x_test1))

5.825

In [96]:
mean_squared_error(y_test1, ada_pred)

10.0

In [97]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 217.9 kB/s eta 0:07:38
   ---------------------------------------- 0.1/99.8 MB 476.3 kB/s eta 0:03:30
   ---------------------------------------- 0.4/99.8 MB 1.8 MB/s eta 0:00:57
   ---------------------------------------- 0.7/99.8 MB 2.6 MB/s eta 0:00:39
   ---------------------------------------- 1.0/99.8 MB 3.4 MB/s eta 0:00:30
   ---------------------------------------- 1.2/99.8 MB 3.5 MB/s eta 0:00:29
    -------------------------------------

In [98]:
#xgboost

In [99]:
import xgboost as xgb

In [100]:
xgb_clf = xgb.XGBClassifier(max_depth = 4, n_estimators = 70, learning_rate = 0.02, n_jobs = -1)

In [101]:
xgb_clf.fit(x_train1, y_train1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=70, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [102]:
mean_squared_error(y_test1, xgb_clf.predict(x_test1))

5.3

In [103]:
# # RandomizedSearchCV for XGB


In [104]:
xgb_clf1 = xgb.XGBClassifier()

In [105]:
xgb_param = {'max_depth': range(3, 10, 2), 'gamma': [0.1, 0.2, 0.3], 'subsample': [0.8, 0.9], 'colsample_bytree': [0.8, 0.9], 'rag_alpha': [1e-2, 0.1, 1]}

In [106]:
xgb_r_grid = RandomizedSearchCV(estimator = xgb_clf1, param_distributions = xgb_param, cv = 5)

In [107]:
xgb_r_grid.fit(x_train1, y_train1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:09] WARNING: C:\buildkite-agent\builds\b

C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [10:56:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)
C:\User

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   param_distributions={'colsample_bytree': [0.8, 0.9],
                                        'gamma': [0.1, 0.2, 0.3],
                                        'max_depth': range(3, 10, 2),
                                        'rag_alpha': [0.01, 0.1, 1],
                                        'subsample': [0.8, 0.9]})

In [108]:
mean_squared_error(y_test1, xgb_r_grid.predict(x_test1))

5.3

In [109]:
xgb_r_grid.best_params_

{'subsample': 0.8,
 'rag_alpha': 0.01,
 'max_depth': 7,
 'gamma': 0.3,
 'colsample_bytree': 0.9}

In [110]:
#compine all the model using SoftVotingClassifier

In [111]:
from sklearn.ensemble import VotingClassifier

In [115]:
voting_estimators = [('dt1_modeel', dt_cv.best_estimator_), ('rf_1model', rf_gd.best_estimator_), ('xgb_clf1', xgb_r_grid.best_estimator_ )]

In [116]:
ensemble_s = VotingClassifier(voting_estimators, voting = 'soft')

In [117]:
soft_voting = ensemble_s.fit(x_train1, y_train1)

C:\Users\LENOVO\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [11:47:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "rag_alpha" } are not used.

  warnings.warn(smsg, UserWarning)


In [118]:
pickle.dump(soft_voting, open('ensemble_assignment_soft_voting.pkl','wb'))